In [1]:
import os
import xarray as xr
import numpy as np

# === Paths ===
combined_file = r"Data - Modelling\Monthly_DB\Combined\Combined_Qld.nc"
output_file = r"Data - Modelling\Monthly_DB\Labelled\Combined_Labeled_Qld.nc"

# === Region-Specific Drought Rules ===
region_rules = {
    "BRB": {"spi_threshold": -1.0, "recovery_period": 3},
    "CHC": {"spi_threshold": -1.0, "recovery_period": 3},
    "CYP": {"spi_threshold": -1.0, "recovery_period": 3},
    "DEU": {"spi_threshold": -1.0, "recovery_period": 5},
    "SEQ": {"spi_threshold": -1.0, "recovery_period": 4},
    "NAN": {"spi_threshold": -1.0, "recovery_period": 3},
    "WET": {"spi_threshold": -1.0, "recovery_period": 2},
    "GDR": {"spi_threshold": -1.0, "recovery_period": 4},
    "NGB": {"spi_threshold": -1.0, "recovery_period": 4},
    "MDD": {"spi_threshold": -1.0, "recovery_period": 3},
    "DLS": {"spi_threshold": -1.0, "recovery_period": 4},
    "SEH": {"spi_threshold": -1.0, "recovery_period": 3},
    "ARN": {"spi_threshold": -1.0, "recovery_period": 3}
}

# === Load the combined file ===
ds_combined = xr.open_dataset(combined_file)

# === Initialize Drought Labels ===
drought = np.zeros_like(ds_combined["spi_1"].values, dtype=np.int8)
consecutive_drought = np.zeros_like(drought, dtype=np.int8)
drought_severity = np.zeros_like(drought, dtype=np.int8)

# === Apply Region-Specific Labeling ===
for region, rules in region_rules.items():
    print(f"✅ Applying rules for region: {region}")
    spi_threshold = rules["spi_threshold"]
    recovery_period = rules["recovery_period"]

    # === Mask for this region ===
    region_mask = (ds_combined["region_code"] == region)
    
    # === Apply Drought Labeling ===
    region_spi = ds_combined["spi_1"].where(region_mask)
    drought_region = (region_spi <= spi_threshold).astype(int)
    
    # === Drought Severity Classification ===
    severity_region = np.zeros_like(region_spi.values)
    severity_region[(region_spi <= -1.0) & (region_spi > -1.5)] = 1  # Moderate Drought
    severity_region[(region_spi <= -1.5) & (region_spi > -2.0)] = 2  # Severe Drought
    severity_region[(region_spi <= -2.0)] = 3                      # Extreme Drought
    
    # === Loop over each pixel to apply consecutive drought logic ===
    for lat in range(drought_region.shape[1]):
        for lon in range(drought_region.shape[2]):
            drought_series = drought_region[:, lat, lon]
            for t in range(1, len(drought_series)):
                if drought_series[t] == 1:
                    previous_droughts = drought_series[max(0, t - recovery_period):t]
                    if previous_droughts.sum() > 0:
                        consecutive_drought[t, lat, lon] = 1

    # === Apply Labels to the Main Arrays ===
    drought[region_mask] = drought_region
    drought_severity[region_mask] = severity_region
    consecutive_drought[region_mask] = np.maximum(consecutive_drought[region_mask], consecutive_drought)

# === Add Labels to the Combined Dataset ===
ds_combined["drought"] = (("time", "lat", "lon"), drought)
ds_combined["drought_severity"] = (("time", "lat", "lon"), drought_severity)
ds_combined["consecutive_drought"] = (("time", "lat", "lon"), consecutive_drought)

# === Save the labeled dataset ===
ds_combined.to_netcdf(output_file)
print(f"✅ Saved labeled dataset: {output_file}")


✅ Applying rules for region: BRB


KeyboardInterrupt: 